# Extract Dynasties, Characters and Titles from Save File

The below code will extract all Dynasties, Characters and Titles from an uncompressed save file and the 00_dynasties.txt file. The save file can be found in the Documents\Paradox Interactive\Crusader Kings II\save games folder and the 00_dynasties.txt file can be found in the C:\Program Files (x86)\Steam\steamapps\common\crusader kings ii\common\dynasties folder. There are two rows in cell 2 that start with "with open". Replace the paths here with the paths to your files. [MongoDB](https://docs.mongodb.com/manual/administration/install-community/) will need to be set up and running for the code to write the data to and for later scripts to read from.

datetime.datetime isn't designed for dates before the 1650's. While it didn't throw any errors for me it may give trouble when comparing dates.

In [1]:
from pymongo import MongoClient
import pymongo
import datetime
import pandas as pd
import re

client = MongoClient()

In [2]:
check_dynasties = False
check_characters = False
check_titles = False

exited_characters = False

exited_titles = False

GAME_END_DATE = None


dyn = {}

dynasties = client.ck2.dynasties
characters = client.ck2.characters
titles = client.ck2.titles


dynasties.delete_many({})
characters.delete_many({})
titles.delete_many({})

#Extract dynasty data from the dynasties game file
with open("../gamefiles/00_dynasties.txt", encoding="cp1252") as dynfile:
        for line in dynfile:
            if len(re.findall(r"^\s*([0-9]+)=", line)) > 0:
                
                if("_id" in dyn.keys()):
                    try:                        
                        dynasties.insert_one(dyn)
                    except pymongo.errors.DuplicateKeyError as e:
                        if ("religion" in dyn.keys()):
                            dynasties.update_one({'_id' : dyn['_id']}, {"$set" : {"religion" : dyn["religion"]}})

                        if ("culture" in dyn.keys()):
                            dynasties.update_one({'_id' : dyn['_id']}, {"$set" : {"culture" : dyn["culture"]}})            
                
                dyn = {}               
                
                #print(re.findall(r"\s*([0-9]+)=", line))
                dyn["_id"] = int(re.findall(r"\s*([0-9]+)=", line)[0])
            
            elif len(re.findall(r"\s*name=\"(.+)\"", line)) > 0:
                dyn["name"] = re.findall(r"\s*name=\"(.+)\"", line)[0]
                
            elif len(re.findall(r"\s*culture=\"(.+)\"", line)) > 0:
                dyn["culture"] = re.findall(r"\s*culture=\"(.+)\"", line)[0]
                
            elif len(re.findall(r"\s*religion=\"(.+)\"", line)) > 0:
                dyn["religion"] = re.findall(r"\s*religion=\"(.+)\"", line)[0]
                
if("_id" in dyn.keys()):
    try:                                                
        dynasties.insert_one(dyn)
    except pymongo.errors.DuplicateKeyError as e:
        if ("religion" in dyn.keys()):
            dynasties.update_one({'_id' : dyn['_id']}, {"$set" : {"religion" : dyn["religion"]}})
        
        if ("culture" in dyn.keys()):
            dynasties.update_one({'_id' : dyn['_id']}, {"$set" : {"culture" : dyn["culture"]}})

#ENCORE!!!            


dyn = {}


#last_char = {}
char = {}
spouse = []


title = {}
holder_list = []
holder_dict = {}
curr_title = None
holder = None
start_date = None
end_date = None
holder_type = None
most_recent_date = None
name = None
odd_title = False

birth_date_hit = False

with open("../ObserverModeSave.ck2", encoding="cp1252") as ck2file:
        for line in ck2file:
            if GAME_END_DATE is None and check_dynasties == False and check_characters == False and check_titles == False and exited_characters == False:
                if len(re.findall(r"\s*date=\"([0-9]+.[0-9]+.[0-9]+)\"", line)) > 0:
                    this_date = re.findall(r"\s*date=\"([0-9]+).([0-9]+).([0-9]+)\"", line)
                    GAME_END_DATE = datetime.datetime(int(this_date[0][0]), int(this_date[0][1]), int(this_date[0][2]))
                    
            
            if len(re.findall(r"^\s*(dynasties=)\s+", line)) > 0:
                check_dynasties = True
                check_characters = False
                check_titles = False
                    
                print("DYNASTIES")
                    
            elif len(re.findall(r"^\s*(character=)\s+", line)) > 0:
                check_dynasties = False
                check_characters = True
                check_titles = False
                
                #FINISH EXECUTION OF LAST DYNASTIES BLOCK
                if("_id" in dyn.keys()):
                    try:                                                
                        dynasties.insert_one(dyn)
                    except pymongo.errors.DuplicateKeyError as e:
                        if ("religion" in dyn.keys()):
                            dynasties.update_one({'_id' : dyn['_id']}, {"$set" : {"religion" : dyn["religion"]}})

                        if ("culture" in dyn.keys()):
                            dynasties.update_one({'_id' : dyn['_id']}, {"$set" : {"culture" : dyn["culture"]}})               
                    
                print("CHARACTERS")
                    
            elif len(re.findall(r"^\s*(delayed_event=)\s*", line)) > 0 and exited_characters == False:
                exited_characters = True                
                check_characters = False
                
                if("b_d" in char.keys()):
                        try:
                            if len(spouse) > 0:
                                char["spouse"] = spouse
                            characters.insert_one(char)
                        except:
                            print("insert failed at top: " + char)            
                
                print("EXITED CHARACTERS")               
                
                    
            elif len(re.findall(r"^\s*(title=)\s+", line)) > 0 and exited_characters == True and check_titles == False and exited_titles == False:
                check_dynasties = False
                check_characters = False
                check_titles = True
                    
                print("TITLES")
            
            elif len(re.findall(r"^\s*(active_ambition=)\s+", line)) > 0 and exited_titles == False:
                if start_date is not None:
                    holder_dict["start_date"] = start_date
                    holder_dict["holder"] = holder
                    if holder_type is not None:
                        holder_dict["type"] = holder_type
                    holder_dict['ruling_as_of'] = GAME_END_DATE
                    holder_dict["reign_length"] = (GAME_END_DATE - start_date).days
                    
                    holder_list.append(holder_dict)            
                if "title" in title.keys():
                    if name is not None:
                        title["name"] = name
                    title["holders"] = holder_list
                    name = None
                    titles.insert_one(title)            
                
                exited_titles = True
                check_titles = False
                print("EXITED TITLES ")
                
            
            if check_dynasties:                
                if len(re.findall(r"^\s*([0-9]+)=", line)) > 0:

                    if("_id" in dyn.keys()):
                        try:                        
                            dynasties.insert_one(dyn)
                        except pymongo.errors.DuplicateKeyError as e:
                            if ("religion" in dyn.keys()):
                                dynasties.update_one({'_id' : dyn['_id']}, {"$set" : {"religion" : dyn["religion"]}})

                            if ("culture" in dyn.keys()):
                                dynasties.update_one({'_id' : dyn['_id']}, {"$set" : {"culture" : dyn["culture"]}})            

                    dyn = {}               

                    #print(re.findall(r"\s*([0-9]+)=", line))
                    dyn["_id"] = int(re.findall(r"\s*([0-9]+)=", line)[0])

                elif len(re.findall(r"\s*name=\"(.+)\"", line)) > 0:
                    dyn["name"] = re.findall(r"\s*name=\"(.+)\"", line)[0]

                elif len(re.findall(r"\s*culture=\"(.+)\"", line)) > 0:
                    dyn["culture"] = re.findall(r"\s*culture=\"(.+)\"", line)[0]

                elif len(re.findall(r"\s*religion=\"(.+)\"", line)) > 0:
                    dyn["religion"] = re.findall(r"\s*religion=\"(.+)\"", line)[0]
                    
            
            
            elif check_characters:
                if len(re.findall(r"^\s*([0-9]+)=", line)) > 0:
                #check b_d is set to prevent inserting first time around. This is because there is no "absolute end" to 
                #a character file, no attribute that is at the end of every character. } is but it appears in a lot of places.
                    if("b_d" in char.keys()):                        
                            try:
                                if len(spouse) > 0:
                                    char["spouse"] = spouse                       
                                characters.insert_one(char)
                            except:
                                print("insert failed at centre: " + char)            
                    #last_char = char
                    char = {}
                    spouse = []

                    #print(re.findall(r"\s*([0-9]+)=", line))
                    char["_id"] = int(re.findall(r"\s*([0-9]+)=", line)[0])

                #birth name
                elif len(re.findall(r"\s*bn=\"(.+)\"", line)) > 0:
                    birth_date_hit = False #putting this here because I'm too lazy to find all the places it should go
                    char["bn"] = re.findall(r"\s*bn=\"(.+)\"", line)[0]
                    
                #Regal name if taken
                elif len(re.findall(r"\s*name=\"(.+)\"", line)) > 0:
                    if birth_date_hit == False:
                        char["taken_name"] = re.findall(r"\s*name=\"(.+)\"", line)[0]

                #isHistorical
                elif len(re.findall(r"\s*hist=(.+)", line)) > 0:
                    char["hist"] = 1

                #isFemale
                elif len(re.findall(r"\s*fem=(.+)", line)) > 0:
                    char["fem"] = 1
                    
                #Turning off detecting regal name if a character has the son_of_evilgod nickname...because that also has a 'name'
                elif len(re.findall(r"^\s*(nick=)\s+", line)) > 0:
                    birth_date_hit = True #turns off check for regal name                   

                #nickname and date of birth
                elif len(re.findall(r"\s*nick=(\S+)\s+b_d=\"([0-9]+.[0-9]+.[0-9]+)\"", line)) > 0:
                    birth_date_hit = True #turns off check for regal name
                    char["nick"] = re.findall(r"\s*nick=(\S+)\s+b_d=\"([0-9]+.[0-9]+.[0-9]+)\"", line)[0][0]
                    this_date = re.findall(r"\s*nick=\S+\s+b_d=\"([0-9]+).([0-9]+).([0-9]+)\"", line)
                    char["b_d"] = datetime.datetime(int(this_date[0][0]), int(this_date[0][1]), int(this_date[0][2])) 

                #birth date
                elif len(re.findall(r"\s*.*\s*b_d=\"([0-9]+.[0-9]+.[0-9]+)\"", line)) > 0:
                    birth_date_hit = True #turns off check for regal name
                    this_date = re.findall(r"\s*b_d=\"([0-9]+).([0-9]+).([0-9]+)\"", line)
                    char["b_d"] = datetime.datetime(int(this_date[0][0]), int(this_date[0][1]), int(this_date[0][2]))
                    
                #death date
                elif len(re.findall(r"\s*d_d=\"([0-9]+.[0-9]+.[0-9]+)\"", line)) > 0:
                    this_date = re.findall(r"\s*d_d=\"([0-9]+).([0-9]+).([0-9]+)\"", line)                
                    char["d_d"] = datetime.datetime(int(this_date[0][0]), int(this_date[0][1]), int(this_date[0][2]))

                #cause of death
                elif len(re.findall(r"\s*c_d=(.+)", line)) > 0:
                    char["c_d"] = re.findall(r"\s*c_d=(.+)", line)[0]

                #killer
                elif len(re.findall(r"\s*killer=([0-9]+)", line)) > 0:
                    char["killer"] = int(re.findall(r"\s*killer=([0-9]+)", line)[0])

                #leige
                elif len(re.findall(r"\s*lge=([0-9]+)", line)) > 0:
                    char["lge"] = int(re.findall(r"\s*lge=([0-9]+)", line)[0])

                #Real father
                elif len(re.findall(r"\s*rfat=([0-9]+)", line)) > 0:
                    char["rfat"] = int(re.findall(r"\s*rfat=([0-9]+)", line)[0])

                #father
                elif len(re.findall(r"\s*fat=([0-9]+)", line)) > 0:
                    char["fat"] = int(re.findall(r"\s*fat=([0-9]+)", line)[0])            

                #mother
                elif len(re.findall(r"\s*mot=([0-9]+)", line)) > 0:
                    char["mot"] = int(re.findall(r"\s*mot=([0-9]+)", line)[0])           

                #spouse
                elif len(re.findall(r"\s*spouse=([0-9]+)", line)) > 0:
                     spouse.append(int(re.findall(r"\s*spouse=([0-9]+)", line)[0]))

                #religion
                elif len(re.findall(r"\s*rel=\"(.+)\"", line)) > 0:
                    char["rel"] = re.findall(r"\s*rel=\"(.+)\"", line)[0]

                #culture
                elif len(re.findall(r"\s*cul=\"(.+)\"", line)) > 0:
                    char["cul"] = re.findall(r"\s*cul=\"(.+)\"", line)[0]

                #graphical culture
                elif len(re.findall(r"\s*g_cul=\"(.+)\"", line)) > 0:
                    char["g_cul"] = re.findall(r"\s*g_cul=\"(.+)\"", line)[0]

                #dynasty
                elif len(re.findall(r"\s*dnt=([0-9]+)", line)) > 0:
                    char["dnt"] = int(re.findall(r"\s*dnt=([0-9]+)", line)[0])

                #isBastard
                elif len(re.findall(r"\s*bstd=(.+)", line)) > 0:
                    char["bstd"] = 1

                #employer? (Only set if character is still living)
                elif len(re.findall(r"\s*emp=([0-9]+)", line)) > 0:
                        char["emp"] = int(re.findall(r"\s*emp=([0-9]+)", line)[0])

                #host? (Only set if character is still living)
                elif len(re.findall(r"\s*host=([0-9]+)", line)) > 0:
                        char["host"] = int(re.findall(r"\s*host=([0-9]+)", line)[0])

                #has Bastards
                elif len(re.findall(r"\s*spawned_bastards=(.+)", line)) > 0:
                    char["spawned_bastards"] = 1
                    
            elif check_titles:
                if len(re.findall(r"^\s*([a-z]{1}_{1}[-_a-z0-9\u00C0-\u017F]*)=\s*", line)) > 0:
                    if start_date is not None:
                        holder_dict["start_date"] = start_date
                        holder_dict["holder"] = holder
                        if holder_type is not None:
                            holder_dict["type"] = holder_type
                        holder_dict['ruling_as_of'] = GAME_END_DATE
                        holder_dict["reign_length"] = (GAME_END_DATE - start_date).days
                        holder_list.append(holder_dict)            
                        if "title" in title.keys():
                            if name is not None:
                                title["name"] = name
                                name = None
                            title["holders"] = holder_list

                        titles.insert_one(title)

                        title = {}
                        holder_list = []
                        holder_dict = {}
                        curr_title = None
                        holder = None
                        start_date = None
                        end_date = None
                        holder_type = None
                        most_recent_date = None
                        name = None
                        odd_title = False

                    title["title"] = re.findall(r"^\s*([a-z]{1}_{1}[-_a-z0-9\u00C0-\u017F]*)=\s*", line)[0]

                    if any(char.isdigit() for char in title["title"]):
                        odd_title = True

                #If a date is found
                if len(re.findall(r"^\s*([0-9]+.[0-9]+.[0-9]+)=", line)) > 0:
                    this_date = re.findall(r"^\s*([0-9]+).([0-9]+).([0-9]+)=", line)
                    date_to_use = datetime.datetime(int(this_date[0][0]), int(this_date[0][1]), int(this_date[0][2]))

                    #Getting past the first run when no date is set
                    if start_date is None:
                        start_date = date_to_use
                        holder_dict["start_date"] = start_date
                    else:
                        most_recent_date = date_to_use



                #Id of the title holder, historical (for rulers before the game start)
                if len(re.findall(r"^\s*holder=([0-9]+)\s*", line)) > 0:

                    if most_recent_date is not None:
                        holder_dict["start_date"] = start_date
                        holder_dict["holder"] = holder
                        holder_dict["end_date"] = most_recent_date
                        holder_dict["reign_length"] = (most_recent_date - start_date).days
                        if holder_type is not None:
                            holder_dict["type"] = holder_type

                        holder_list.append(holder_dict.copy())

                        holder_dict = {}
                        start_date = most_recent_date
                        end_date = None
                        holder = None
                        holder_type = None

                    if start_date is not None:
                        holder = int(re.findall(r"^\s*holder=([0-9]+)\s*", line)[0])

                #Where title has lapsed - no holder
                if len(re.findall(r"^\s*(holder=\"0\")\s*", line)) > 0:            

                    if most_recent_date is not None:
                        holder_dict["start_date"] = start_date
                        holder_dict["holder"] = holder
                        holder_dict["end_date"] = most_recent_date
                        holder_dict["reign_length"] = (most_recent_date - start_date).days
                        if holder_type is not None:
                            holder_dict["type"] = holder_type

                        holder_list.append(holder_dict.copy())

                        holder_dict = {}
                        start_date = most_recent_date
                        end_date = None
                        holder = None
                        holder_type = None

                    holder = 0


                #Id of the title holder (where who and type are set)
                elif len(re.findall(r"^\s*who=([0-9]+)\s*", line)) > 0:

                    if most_recent_date is not None:
                        holder_dict["start_date"] = start_date
                        holder_dict["holder"] = holder
                        holder_dict["end_date"] = most_recent_date                        
                        holder_dict["reign_length"] = (most_recent_date - start_date).days
                        
                        if holder_type is not None:
                            holder_dict["type"] = holder_type

                        holder_list.append(holder_dict.copy())

                        holder_dict = {}
                        start_date = most_recent_date
                        end_date = None
                        holder = None
                        holder_type = None

                    holder = int(re.findall(r"^\s*who=([0-9]+)\s*", line)[0])


                #type (election, inherited etc)
                elif len(re.findall(r"^\s*type=(\w+)\s*", line)) > 0:
                    if holder is not None:
                        holder_type = re.findall(r"^\s*type=(\w+)\s*", line)[0].strip()

                #some titles are generated during the game (id d_dyn_123456) such as revolts and patrician families.  
                #These contain a number of "name=" rows (wars etc). The row that is wanted may contain unicode
                elif len(re.findall(r"^\s*name=\"([-a-zA-Z\s\u00C0-\u017F]*)\"\s*", line)) > 0:
                    if "title" in title.keys() is not None:
                        if odd_title and len(holder_list) <= 0:
                            name = re.findall(r"^\s*name=\"([-a-zA-Z\s\u00C0-\u017F]*)\"\s*", line)[0]
                            odd_title = False
                            
                elif len(re.findall(r"^\s*(active=no)\s*", line)) > 0:
                    title["inActive"] = 1
                            

DYNASTIES
CHARACTERS
EXITED CHARACTERS
TITLES
EXITED TITLES 


The cell below adds two arrays to each character, one containing that characters "fat" value (father) and mother while the other one holds there "rfat" (real father) value and the mother. If the rfat value is null then the fat value will be included instead. For each character the first array will hold their stated father and the second will hold their actual father.

In [3]:
#add parents and rparents field to use with $graphlookup for full descent. rparents contains rfat (real father) if it is set, else fat.
characters.aggregate(
    [
        { "$addFields": { 
            "parents": ["$fat", "$mot"],
            "rparents": [{"$ifNull" : ["$rfat", "$fat"] }, "$mot"] 
        }},
        { "$out": "characters" }
    ]
)